In [1]:
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import os 
%matplotlib inline

In [2]:
file = "D:\\Learn\\Uni\\ML\\CapstoneProject\\data\\healthcare-dataset-stroke-data.csv"
df = pd.read_csv(file)

In [3]:
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())

Pre Processing

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [6]:
LE = LabelEncoder()
df['gender'] = LE.fit_transform(df['gender'])
df['ever_married'] = LE.fit_transform(df['ever_married'])
df['work_type'] = LE.fit_transform(df['work_type'])
df['Residence_type'] = LE.fit_transform(df['Residence_type'])
df['smoking_status'] = LE.fit_transform(df['smoking_status'])

In [7]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,28.891862,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [8]:
X = df.iloc[:,1:-1].values
y = df.iloc[:,-1].values

print(X.shape, y.shape, sep='\n')

(5104, 10)
(5104,)


In [9]:
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,5,9])],remainder='passthrough')
X = ct.fit_transform(X)

In [10]:
print('Example of Processed Data:')
print(X[0])

Example of Processed Data:
[  0.     1.     0.     0.     0.     1.     0.     0.     0.     1.
   0.     0.    67.     0.     1.     1.     1.   228.69  36.6 ]


Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=8)

In [13]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#displays the amount of samples in each dataset
print('Samples in Train Set:', len(X_train))
print('Samples in Validation Set:', len(X_val))
print('Samples in Test Set:', len(X_test))

(3572, 19) (766, 19) (3572,) (766,)
Samples in Train Set: 3572
Samples in Validation Set: 766
Samples in Test Set: 766


Smote

In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
sm = SMOTE(random_state=8)
X_bal, y_bal = sm.fit_resample(X_train, y_train)


In [16]:
print(X_bal.shape)

#Displays perccent of each class
print('Initial Dataset')
print('Percent "stroke":', y_train.sum()/len(y_train))
print('Balanced Dataset')
print('Percent "stroke":', y_bal.sum()/len(y_bal))

(6804, 19)
Initial Dataset
Percent "stroke": 0.04759238521836506
Balanced Dataset
Percent "stroke": 0.5


Creating the Neural Networ

In [17]:
import tensorflow
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
inputs = keras.Input(shape=(19,))
a = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.6)(a)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.concatenate([a, x])
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.6)(x)
output = layers.Dense(1, activation='sigmoid')(x)

nn = keras.Model(inputs, output, name="stroke_predictor")

In [19]:
nn.compile(optimizer=tensorflow.optimizers.Adam(learning_rate=0.0001),
            loss='binary_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

#Trains the neural network
history = nn.fit(X_bal, y_bal, epochs=1000, callbacks=[early_stopper], validation_data=(X_val, y_val), verbose=0)

Test Model

In [20]:
from sklearn.metrics import accuracy_score, recall_score ,precision_score, f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [21]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)
accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10)
print(accuracies)

NameError: name 'model' is not defined

In [ ]:
test_preds = nn.predict(X_test)
threshold = 0.25
test_preds = [1. if i > threshold else 0. for i in test_preds]
test_acc = accuracy_score(y_test, test_preds)
test_rec = recall_score(y_test, test_preds)
test_pre = precision_score(y_test, test_preds)
print('Test Set Metrics')
print('Model Accuracy:', test_acc)
print('Model Recall:', test_rec)
print('Model Precision:', test_pre)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Test Set Metrics
Model Accuracy: 0.3754889178617992
Model Recall: 1.0
Model Precision: 0.07528957528957529
